# Model the Data
Given a set of scalar data, you can model it with a suitable pdf. This exercise will help you learn the following:
- How would you choose the best model to fit the given data?
- How would you estimate the model parameters from the given data?
- Given a model, how do you sample new data from it?

Note: You are allowed to use only numpy and matplotlib libraries. No ML library.

In [1]:
import numpy as np
np.random.seed(111)

In [2]:
def data2gaussian(S):
    '''
    Return optimal parameters - (mu,sigma)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        sigma: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    N = len(S)
    sample_sum=0
    for i in range(N):
        sample_sum+=S[i]
    mu = sample_sum*(1.0/N)
    sample_sq_sum=0
    for i in range(N):
        sample_sq_sum+=(S[i]-mu)*(S[i]-mu)
    sigma = np.power(sample_sq_sum*(1.0/N),0.5)
    return mu, sigma

In [3]:
def test_data2gaussian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, sigma = data2gaussian(S)
    print(mu,sigma)
    assert isinstance(mu, (int, float))
    assert isinstance(sigma, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2gaussian()

0.06000000000000001 0.19595917942265426
Test passed 👍


In [4]:
def data2laplacian(S):
    '''
    Return optimal parameters - (mu,b). See https://en.wikipedia.org/wiki/Laplace_distribution
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    mu = np.median(S)
    sample_abs_sum=0
    for i in range(len(S)):
        sample_abs_sum+=np.abs(S[i]-mu)
    b = (2.0/len(S))*sample_abs_sum

    return mu, b

In [5]:
def test_data2laplacian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, b = data2laplacian(S)
    print(mu,b)
    assert isinstance(mu, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2laplacian()

0.0 0.28
Test passed 👍


In [6]:
def data2uniform(S):
    '''
    Return optimal parameters - (a,b)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        a: float
        b: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    a = np.amin(S,axis=0)
    b = np.amax(S,axis=0)
    return a, b

In [7]:
def test_data2uniform(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    a, b = data2uniform(S)
    print(a,b)
    assert isinstance(a, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2uniform()

-0.2 0.4
Test passed 👍


In [8]:
def data2model(S):
    '''
    Inputs:
        S: np array of shape (Ns,). These are scalar samples of a random variable.
    Outputs:
        modelName: return one out of these - 'gaussian', 'laplacian', or 
                   'uniform' which best models the data
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS
    S1 = np.sort(S,axis=0)
    p1 = []
    samples = []
    i=0
    while(i<len(S1)):
        samples.append(S1[i])
        cnt=1
        if(i+1<len(S1) and S1[i]==S1[i+1]):
            cnt+=1
            i+=1
        p1.append(cnt*(1.0/len(S1)))
        i+=1
    a,b = data2uniform(S)
    uniform = []
    for i in range(len(p1)):
        uniform.append(1.0/(b-a))
    mu,sigma=data2gaussian(S)
    gaussian = (1.0/np.power(2*np.pi*sigma*sigma,0.5))*np.exp(-(0.5/(sigma*sigma))*((np.asarray(samples)-mu)**2))
    mu,b = data2laplacian(S)
    laplacian = (0.5/b)*np.exp(-(1.0/b)*np.abs(samples-mu))

    #using kldivergence as the metric for best model prediction
    loss = np.inf
    modelName=''
    kldivergence=0
    
    for i in range(len(p1)):
        kldivergence+=p1[i]*np.log10(p1[i]*(1.0/uniform[i]))

    if(kldivergence<loss):
        loss = kldivergence
        modelName='uniform'
    
    kldivergence=0
    
    for i in range(len(p1)):
        kldivergence+=p1[i]*np.log10(p1[i]*(1.0/gaussian[i]))
    
    if(kldivergence<loss):
        loss = kldivergence
        modelName='gaussian'
    
    kldivergence=0
    
    for i in range(len(p1)):
        kldivergence+=p1[i]*np.log10(p1[i]*(1.0/laplacian[i]))
    
    if(kldivergence<loss):
        loss = kldivergence
        modelName='laplacian'
    return modelName

In [9]:
def test_data2model(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    modelName = data2model(S)
    assert modelName in ['gaussian', 'laplacian', 'uniform']
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2model()

Test passed 👍


In [10]:
def sampleGMM(pi, mu, sigma, Ns=1):
    '''
    Inputs:
        pi: np.array of shape (K,), p(z_k)
        mu: np.array of shape (K,), mu of kth gaussian
        sigma: np.array of shape (K,), sigma of kth gaussian
        Ns: int, number of samples
    Outputs:
        S: np.array of shape (Ns,), samples from the GMM
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS
    mixture_idx = np.random.choice(len(pi), size=Ns, replace=True, p=pi)
    S=[]
    for i in range(Ns):
        s = np.random.normal(mu[mixture_idx[i]],sigma[mixture_idx[i]],1)
        S.append(s[0])
    S=np.asarray(S)
    S=S.reshape(-1)
    return S

In [11]:
def test_sampleGMM(): # checks formatting only
    pi = [0.3,0.7]
    mu = [-1.1, 1.3]
    sigma = [1.5, 0.4]
    Ns = 5
    S = sampleGMM(pi, mu, sigma, Ns)
    assert S.shape==(5,)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_sampleGMM()

Test passed 👍


In [12]:
def make_spd_matrix(K,Na):
    cov = np.zeros((K,Na,Na))
    for i in range(K):
        A = np.random.randn(Na,Na)
        A = 0.5*(A+A.T)
        A = A + Na*np.eye(Na,dtype=np.float64)
        cov[i]=A
    return cov

def data2GMM(S, K):
    '''
    Return optimal parameters - (pi,mu,sigma)
    Inputs:
        S: np array of shape (Ns,Na). These are samples of a random variable. Na can be 1, 2 or 3
    Outputs:
        pi: np array of shape (K,)
        mu: np array of shape (K,Na)
        sigma: np array of shape (K,Na,Na)
    '''

    ### WRITE YOUR CODE HERE - 15 MARKS
    pi = np.ones(K, dtype=np.float64)*(1.0/K)
    mu = np.random.choice(S.flatten(), (K,S.shape[1]))
    sigma = make_spd_matrix(K,S.shape[1])
    gamma = np.zeros((S.shape[0],K))
    N = S.shape[0]
    D = S.shape[1]
    i=0
    log_likeli=0
    for n in range(N):
        sums=0
        for k in range(K):
            sums+=pi[k]*(1.0/(np.linalg.det(sigma[k])*np.power(2*np.pi,D/2)))*np.exp(-0.5*np.dot(S[n]-mu[k],np.dot(np.linalg.inv(sigma[k]),(S[n]-mu[k]).T)))
        log_likeli+=np.log(sums)
    
    prev=log_likeli
    eps=1e-8
    tol=1e-2
    while(True):
        
        #gamma updation
        for n in range(N):
            gammas=0
            for k in range(K):
                gammas+=pi[k]*(1.0/(np.linalg.det(sigma[k])*np.power(2*np.pi,D/2)))*np.exp(-0.5*np.dot(S[n]-mu[k],np.dot(np.linalg.inv(sigma[k]),(S[n]-mu[k]).T)))
            for k in range(K):
                gamma[n][k] =(1.0/(gammas+eps))*(pi[k]*(1.0/(np.linalg.det(sigma[k])*np.power(2*np.pi,D/2)))*np.exp(-0.5*np.dot(S[n]-mu[k],np.dot(np.linalg.inv(sigma[k]),(S[n]-mu[k]).T))))
                
        #mean updation
        for k in range(K):
            sums=np.zeros((1,D))
            Nk=0
            for n in range(N):
                sums+=gamma[n][k]*S[n]
                Nk+=gamma[n][k]
            mu[k] = sums*(1.0/(Nk+eps))
            
        #sigma updation
        for k in range(K):
            sums=np.zeros((D,D))
            Nk=0
            for n in range(N):
                d = np.reshape(S[n]-mu[k],(1,3))
                sums+=gamma[n][k]*np.dot(d.T,d)
                Nk+=gamma[n][k]
            sigma[k]=sums*(1.0/(Nk+eps))
            
        #pi updation
        for k in range(K):
            Nk=0
            for n in range(N):
                Nk+=gamma[n][k]
            pi[k]=Nk*(1.0/N)
            
        #evaluate the log likelihood
        log_likeli=0
        for n in range(N):
            sums=0
            for k in range(K):
                sums+=pi[k]*(1.0/(np.linalg.det(sigma[k])*np.power(2*np.pi,D/2)))*np.exp(-0.5*np.dot(S[n]-mu[k],np.dot(np.linalg.inv(sigma[k]),(S[n]-mu[k]).T)))
            log_likeli+=np.log(sums)
        print(log_likeli) 
        if(np.abs(prev-log_likeli)<tol):
            break
        prev=log_likeli
        
    pi=pi.reshape(-1)
    return pi, mu, sigma

In [15]:
def test_data2GMM(): # checks formatting only
    S = np.random.random((10,3))
    pi, mu, sigma = data2GMM(S,2)
    assert pi.shape==(2,)
    assert mu.shape==(2,3)
    assert sigma.shape==(2,3,3)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2GMM()

35.564564418952074
35.591571650893364
35.74093097964408
36.496745452893364
38.51629400304229
40.390356129091366
41.91353967023679
43.81489348806791
46.311618058345594
52.5381769159801
57.31304550980013
59.52885056800288
59.61021311546592
59.60497099363863
Test passed 👍
